<a href="https://colab.research.google.com/github/RoetGer/decisions-under-uncertainty/blob/main/data_science_and_stochastic_programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cvxpy
!pip install cvxstoc

     |████████████████████████████████| 389kB 6.0MB/s 
  Created wheel for pymc: filename=pymc-2.3.8-cp37-cp37m-linux_x86_64.whl size=1352895 sha256=d6067e4e25ffa8a1b271fb9ca0d77c4edc7220a0ff0e0433cbcf2b57ebd3a8e9
  Stored in directory: /root/.cache/pip/wheels/0b/a8/e7/8f3ba91a39294d538a92db052fd1fcba1fca74a58c8b022026
Successfully built pymc


# Data Science and Stochastic Programming

In this notebook we explore, how stochastic programming can be used to incorporate uncertainty stemming from data science models into our decision making process.

In [16]:
import cvxstoc
from cvxstoc import NormalRandomVariable, expectation, prob
from cvxpy import Maximize, Problem
from cvxpy.expressions.variable import Variable
import numpy as np
import pymc

# Samples to be taken
num_samples = 100

# Create problem data.
n = 10
mu = np.zeros(n)
Sigma = 0.1*np.eye(n)
p = NormalRandomVariable(mu, Sigma)
alpha = -1
beta = 0.05

# Create and solve stochastic optimization problem.
x = Variable(n)
p = Problem(
    Maximize(expectation(x.T*p, num_samples=num_samples)),
    [
      x >= 0, 
      x.T*np.ones(n) == 1,
      prob(x.T*p <= alpha, num_samples=num_samples) <= beta
    ]
)

p.solve()

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


0.062432367615797305

In [ ]:
?expectation

In [3]:
from cvxstoc import RandomVariable, RandomVariableFactory

In [ ]:
??cvxstoc

In [ ]:
??RandomVariable

In [4]:
??NormalRandomVariable

In [6]:
??RandomVariableFactory

In [5]:
??RandomVariableFactory.create_normal_rv

In [ ]:
pymc_rv = pymc.stochastic_from_data(
    name="Empirical", 
    data=np.random.normal(size=(100,)), 
    lower=-np.inf, 
    upper=np.inf)

In [ ]:
pymc_rv.random()

0.46706672849725195

In [ ]:
??pymc.Normal

In [ ]:
pymc.Normal(name="blub", mu=2., tau=1., size=(3,))

<pymc.distributions.new_dist_class.<locals>.new_class 'blub' at 0x7f7c9249aad0>

In [ ]:
??RandomVariable

In [11]:
def EmpiricalRandomVariable(samples, mu):
    return create_empirical_rv(samples, mu)

def create_empirical_rv(samples, mu, lower=-np.inf, upper=np.inf):
    rv_name = "empiricial_placeholder"

    rv_pymc = pymc.stochastic_from_data(
        name="Empirical", 
        data=samples, 
        lower=lower, 
        upper=upper)
    
    metadata = {
      "mu": mu
    }

    return RandomVariable(rv=rv_pymc, metadata=metadata)

samples = np.random.normal(size=(100,))
mu = np.mean(samples)

erv = EmpiricalRandomVariable(samples, mu)


In [12]:
del p

In [23]:
p = NormalRandomVariable(mu, Sigma)
x.T*p
expectation(x.T*p, num_samples=num_samples)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


Expression(AFFINE, UNKNOWN, (10,))

In [24]:
p = EmpiricalRandomVariable(samples, np.mean(samples))
x.T*p
expectation(x*p, num_samples=num_samples)

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


Expression(AFFINE, UNKNOWN, (1,))

In [34]:
p = EmpiricalRandomVariable(samples, np.mean(samples))
alpha = -1
beta = 0.05
n =  1

# Create and solve stochastic optimization problem.
x = Variable(n, name="x")
p = Problem(
    Maximize(expectation(x*p, num_samples=num_samples)),
    [
      x >= 0, 
     # x*np.ones(n) == 1,
      prob(x*p <= alpha, num_samples=num_samples) <= beta
    ]
)

p.solve()

/usr/local/lib/python3.7/dist-packages/pymc/MCMC.py:81: UserWarning: Instantiating a Model object directly is deprecated. We recommend passing variables directly to the Model subclass.
  warnings.warn(message)


-3.991545276504316e-12

In [35]:
np.mean(samples)

-0.2177907409688715

In [36]:
import scipy as scp

scp.random.normal

In [33]:
for variable in p.variables():
    print("Variable %s: value %s" % (variable.name(), variable.value))

Variable x: value [2.42765387e-11]
Variable var48963: value 1.012806553248664
